In [1]:
import os
import numpy as np
import pandas as pd
from utils.utils import *
from utils.kabsch2D import * 
from scipy import stats

In [3]:
def create_rotation_df(subj, rois, sess):
    cols = ['source', 'base', 'target', 'U', 'distance']
    rotations = np.zeros(((len(rois) * (len(rois)-1)) * 2, len(cols)), dtype=object)
    i = 0 
    for roi_source in rois.keys():
        mds_source = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sess}_{roi_source}_mds.npy', allow_pickle=False)
        for roi_target in rois.keys():
            if roi_source != roi_target:
                for j in range(2):
                    mds_target = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sess}_{roi_target}_mds.npy', allow_pickle=False)
                    # flip the target when j is at 1, does nothing when j = 0 
                    rotations[i, 1] = roi_target
                    if j == 1:
                        mds_target = np.dot(mds_target, np.array([[-1, 0], [0, 1]]))
                        roi_target = roi_target + "_flipped"
                    U, t= kabsch2D(mds_source, mds_target, translate=True )
                    rotations[i, 0] = roi_source
                    rotations[i, 2] = roi_target
                    rotations[i, 3] = U
                    rotated_source = rotate(mds_source, U)
                    rotations[i, 4] = avg_distance(rotated_source, mds_target, t)
                    i += 1
    df = pd.DataFrame(rotations, columns=cols)
    df = df.astype({'distance': 'float32'}) # needed for later indexing
    return df

def filter_df(df):
 # get the index of the lowest error for each pair in each ROI
    indx_to_keep = df.groupby(['source', 'base'])['distance'].idxmin()
    df_filtered = df.loc[indx_to_keep]
    return df




In [8]:
subj_list = [f'subj0{i}' for i in range(1,9)]
df_01 = filter_df(create_rotation_df(subj_list[0], rois, sessions[0]))
df_01

FileNotFoundError: [Errno 2] No such file or directory: '/home/stan/thesis-repo/data/MDS/subj01/subj01_40_V1_mds_betas_train.npy'

In [2]:
def create_distance_df(subj, rois, sess, rotated_df):
    cols = rois.keys
    rotations = np.zeros((len(cols), len(cols)), dtype=object)
    i = 0 
    for roi_source in rois.keys():
        mds_source = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sess}_{roi_source}_mds_betas_train.npy', allow_pickle=False)
        for roi_target in rois.keys():
            if roi_source != roi_target:
                for j in range(2):
                    mds_target = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sess}_{roi_target}_mds_betas_train.npy', allow_pickle=False)
                    # flip the target when j is at 1, does nothing when j = 0 
                    rotations[i, 1] = roi_target
                    if j == 1:
                        mds_target = np.dot(mds_target, np.array([[-1, 0], [0, 1]]))
                        roi_target = roi_target + "_flipped"
                    U, t= kabsch2D(mds_source, mds_target, translate=True )
                    rotations[i, 0] = roi_source
                    rotations[i, 2] = roi_target
                    rotations[i, 3] = U
                    rotated_source = rotate(mds_source, U)
                    rotations[i, 4] = avg_distance(rotated_source, mds_target)
                    i += 1
    df = pd.DataFrame(rotations, columns=cols)
    df = df.astype({'distance': 'float32'}) # needed for later indexing
    return df